In [1]:
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration
import torch

/nas/home/amandeep/miniconda3/envs/pyserini/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")

retriever = RagRetriever.from_pretrained(

    "facebook/rag-token-nq", index_name="exact", use_dummy_dataset=False

)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 28.39it/s]

















































































































Generating train split:  29%|██████████████████████████████████████████████▌                                                                                                                 | 6118091/21015300 [1:55:21<4:31:53, 913.19 examples/s]

In [4]:
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.92G/1.92G [00:22<00:00, 91.7MB/s]
Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ra

In [14]:
questions = ["When did Barack Obama get married?", 
             "When did Joe Biden get married?", 
             "When did Joe Biden and Neilia get married?",
             "When did Sachin Tendulkar get married?"]

In [15]:
for question in questions:
    inputs = tokenizer(question, return_tensors="pt")
    input_ids = inputs["input_ids"]

    question_hidden_states = model.question_encoder(input_ids)[0]
    docs_dict = retriever(input_ids.numpy(), question_hidden_states.detach().numpy(), return_tensors="pt")
    doc_scores = torch.bmm(
        question_hidden_states.unsqueeze(1), docs_dict["retrieved_doc_embeds"].float().transpose(1, 2)
    ).squeeze(1)

    generated = model.generate(
        context_input_ids=docs_dict["context_input_ids"],
        context_attention_mask=docs_dict["context_attention_mask"],
        doc_scores=doc_scores,
    )
    generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)
    print(generated_string)

[' october 3, 1992']
[' june 17, 1977']
[' january 28, 1956']
[' 24 may 1995']


In [21]:
import json
from typing import List

In [23]:
def answer_question(question: str) -> List[str]:
    inputs = tokenizer(question, return_tensors="pt")
    input_ids = inputs["input_ids"]

    question_hidden_states = model.question_encoder(input_ids)[0]
    docs_dict = retriever(input_ids.numpy(), question_hidden_states.detach().numpy(), return_tensors="pt")
    doc_scores = torch.bmm(
        question_hidden_states.unsqueeze(1), docs_dict["retrieved_doc_embeds"].float().transpose(1, 2)
    ).squeeze(1)

    generated = model.generate(
        context_input_ids=docs_dict["context_input_ids"],
        context_attention_mask=docs_dict["context_attention_mask"],
        doc_scores=doc_scores,
    )
    generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)
    return generated_string

In [18]:
questions = json.load(open('marriages_in_wikidata.json'))

In [19]:
questions[0]

{'q': 'when did Imra Ardeen get married to Mon-El?',
 'a': '3100-01-01',
 'n1': 'Q105432158',
 'n2': 'Q96108374',
 'n1_label': 'Imra Ardeen',
 'n2_label': 'Mon-El'}

In [24]:
dpr_answers = open('marriages_wikidata.dpr.answers.rag.generation.jl', 'w')

In [ ]:
count = 1
for question in questions:
    if count % 10000 == 0:
        print(count)
    dpr_answer = answer_question(question['q'])
    question['dpr_a'] =  dpr_answer
    dpr_answers.write(json.dumps(question))
    dpr_answers.write('\n')
    count += 1

/nas/home/amandeep/miniconda3/envs/pyserini/lib/python3.8/site-packages/transformers/generation_utils.py:2129: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
